In [290]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
data = None
test_data = None
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if filename == 'train.csv':
            data = pd.read_csv(path)
        if filename == 'test.csv':
            test_data = pd.read_csv(path)
        if filename == 'store.csv':
            store_data = pd.read_csv(path)
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [291]:
# Join train set and store set, test set and store set, inner refers to using the common keys
# on refers to the label
new_train = data.merge(store_data, how='inner', on='Store')
new_test = test_data.merge(store_data, how='inner', on='Store')

In [292]:
new_train['Date'] = pd.to_datetime(new_train['Date'])
new_train['Year'] = new_train.Date.dt.year
new_train['Month'] = new_train.Date.dt.month
new_train['Day'] = new_train.Date.dt.day

new_test['Date'] = pd.to_datetime(new_test['Date'])
new_test['Year'] = new_train.Date.dt.year
new_test['Month'] = new_test.Date.dt.month
new_test['Day'] = new_test.Date.dt.day

In [293]:
new_train["CompetitionDistance"]=new_train["CompetitionDistance"].fillna(99999)
new_test["CompetitionDistance"]=new_test["CompetitionDistance"].fillna(99999)

In [294]:
 # convert assortment and storetype and stateholiday in training data to int
cleanup = {"Assortment": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup, inplace=True)
new_train['Assortment'] = new_train['Assortment'].astype(int)

cleanup2 = {"StoreType": {"a":0, "b":1, "c":2, "d":3}}
new_train.replace(cleanup2, inplace=True)
new_train['StoreType'] = new_train['StoreType'].astype(int)

cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup3, inplace=True)
new_train['StateHoliday'] = new_train['StateHoliday'].astype(int)

In [295]:
 # convert assortment and storetype in testing data to int
cleanup = {"Assortment": {"a":0, "b":1, "c":2}}
new_test.replace(cleanup, inplace=True)
new_test['Assortment'] = new_test['Assortment'].astype(int)

cleanup2 = {"StoreType": {"a":0, "b":1, "c":2, "d":3}}
new_test.replace(cleanup2, inplace=True)
new_test['StoreType'] = new_test['StoreType'].astype(int)

cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup3, inplace=True)
new_train['StateHoliday'] = new_train['StateHoliday'].astype(int)

In [296]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 21 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  int64         
 8   SchoolHoliday              1017209 non-null  int64         
 9   StoreType                  1017209 non-null  int64         
 10  Assortment                 1017209 non-null  int64         
 11  CompetitionDistance        1017209 no

In [297]:
 # Null value exists in open, fill with one, cuz stores would open except from holidays
new_train['Open'] = new_train['Open'].fillna(1)
new_train['Open'] = new_train['Open'].astype(int)


new_test['Open'] = new_test['Open'].fillna(1)
new_test['Open' ]= new_test['Open'].astype(int)

In [298]:
 # fill nan with month 0, years in the far future
new_train['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
new_train['CompetitionOpenSinceYear'].fillna(2100, inplace=True)

new_train['Promo2SinceWeek'].fillna(1, inplace=True)
new_train['Promo2SinceYear'].fillna(2100, inplace=True)
new_train['PromoInterval'].fillna(0, inplace=True)

new_test['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
new_test['CompetitionOpenSinceYear'].fillna(2100, inplace=True)

new_test['Promo2SinceWeek'].fillna(1, inplace=True)
new_test['Promo2SinceYear'].fillna(2100, inplace=True)
new_test['PromoInterval'].fillna(0, inplace=True)

In [299]:
 # combine competition month and year and promo week year to form days
new_train['CompetitionDays'] = 365 * (new_train['Year'] - new_train['CompetitionOpenSinceYear']) + new_train['Month'] - new_train['CompetitionOpenSinceMonth']

new_train['PromoOpenDays'] = 365 * (new_train['Year'] - new_train['Promo2SinceYear']) + ((new_train['Day'] - 7 * new_train['Promo2SinceWeek']))

new_test['CompetitionDays'] = 365 * (new_test['Year'] - new_test['CompetitionOpenSinceYear']) + new_test['Month'] - new_test['CompetitionOpenSinceMonth']

new_test['PromoOpenDays'] = 365 * (new_test['Year'] - new_test['Promo2SinceYear']) + ((new_test['Day'] - 7 * new_test['Promo2SinceWeek']))

In [300]:
new_train.drop(columns = ['CompetitionOpenSinceMonth'], inplace = True)
new_train.drop(columns =['CompetitionOpenSinceYear'], inplace = True)
new_train.drop(columns =['Promo2SinceWeek'], inplace = True)
new_train.drop(columns =['Promo2SinceYear'], inplace = True)
new_train.drop(columns =['Date'], inplace = True)

In [301]:
new_train = new_train.query('Open==1')
new_test = new_test.query('Open==1')

In [302]:
# Normalize
"""
new_train_plus = new_train
columns = ['Sales', 'CompetitionDistance', 'CompetitionDays', 'PromoOpenDays', "Customers"]
for col in columns:
    max_value = np.max(new_train_plus[col])
    min_value = np.min(new_train_plus[col])
    new_train_plus[col] = (new_train_plus[col] - min_value) / (max_value - min_value)

new_train = new_train_plus
"""
new_train.drop(columns=['Customers'],inplace = True)


In [303]:
new_train.dtypes

Store                    int64
DayOfWeek                int64
Sales                    int64
Open                     int64
Promo                    int64
StateHoliday             int64
SchoolHoliday            int64
StoreType                int64
Assortment               int64
CompetitionDistance    float64
Promo2                   int64
PromoInterval           object
Year                     int64
Month                    int64
Day                      int64
CompetitionDays        float64
PromoOpenDays          float64
dtype: object

In [304]:
new_train = pd.get_dummies(new_train, columns = ['PromoInterval'])
new_test = pd.get_dummies(new_test, columns = ['PromoInterval'])

In [305]:
new_train.dtypes

Store                               int64
DayOfWeek                           int64
Sales                               int64
Open                                int64
Promo                               int64
StateHoliday                        int64
SchoolHoliday                       int64
StoreType                           int64
Assortment                          int64
CompetitionDistance               float64
Promo2                              int64
Year                                int64
Month                               int64
Day                                 int64
CompetitionDays                   float64
PromoOpenDays                     float64
PromoInterval_0                     uint8
PromoInterval_Feb,May,Aug,Nov       uint8
PromoInterval_Jan,Apr,Jul,Oct       uint8
PromoInterval_Mar,Jun,Sept,Dec      uint8
dtype: object

In [306]:
x = new_train.drop(["Sales"], axis = 1)
y = new_train["Sales"]

In [307]:
x = x.to_numpy()
y = y.to_numpy()

In [308]:
len(x[0])

19

In [309]:
import torch
from torch import nn

In [310]:
class Autoencoder(torch.nn.Module):
    
    def __init__(self, n_inp, n_hidden, name):
        super().__init__()
        
        print(n_inp, "->", n_hidden)
        
        self.hidden = nn.Linear(in_features=n_inp, out_features=n_hidden)
        print(self.hidden)
        self.output = nn.Linear(in_features=n_hidden, out_features=n_inp)
        print(self.output)
        print(n_inp, "->", n_hidden)
        self.is_learning = True
    
    def set_learning(self, value):
        self.is_learning = value
        
    def forward(self, x, function=torch.relu):
        
        output = function(self.hidden(x))
        
        if self.is_learning:
            output = function(self.output(output))
        
        return output

In [311]:
def trainAutoencoder(epochs: int, size: int, _input = x):
    
        layer = Autoencoder(len(_input[0]), size, epochs)

        optimizer = torch.optim.Adam(layer.parameters(),lr=0.0001)

        # mean-squared error loss
        criterion = nn.MSELoss()

        for epoch in range(epochs):
            loss = 0

            train_loader = torch.utils.data.DataLoader(_input, batch_size=10000, shuffle=True, num_workers=4, pin_memory=True)

            for batch_features in train_loader:
                
                batch_features = batch_features.view(-1, len(_input[0]))
                optimizer.zero_grad()
                outputs = layer(batch_features.float())
                train_loss = criterion(outputs, batch_features.float())
                train_loss.backward()
                optimizer.step()
                loss += train_loss.item()

            loss = loss / len(train_loader)
            print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
        layer.set_learning(False)
        return layer

In [312]:
def feedforward(models: [], x):
    
    output = torch.tensor(x).float()

    for model in models:
        output1 = model(output)
        output = output1
        
    return output

In [313]:
def trainLayers(epochs: [], size_of_layer: [], _input):
    assert len(epochs) == len(size_of_layer)
    models = []
    print(size_of_layer[0])
    hidden_layer = trainAutoencoder(epochs[0], size_of_layer[0], _input)
    models.append(hidden_layer)
    
    for index, size in enumerate(size_of_layer):
        if index == 0:
            continue
        h_layer_2 = trainAutoencoder(epochs[index], size_of_layer[index], feedforward(models, _input).detach())
        models.append(h_layer_2)
    
    return models

models = trainLayers([2, 2, 2], [10, 5, 1], x)

10
19 -> 10
Linear(in_features=19, out_features=10, bias=True)
Linear(in_features=10, out_features=19, bias=True)
19 -> 10
epoch : 1/2, loss = 52460988.376471
epoch : 2/2, loss = 51502040.658824
10 -> 5
Linear(in_features=10, out_features=5, bias=True)
Linear(in_features=5, out_features=10, bias=True)
10 -> 5
epoch : 1/2, loss = 7325256.041176
epoch : 2/2, loss = 7256154.676471
5 -> 1
Linear(in_features=5, out_features=1, bias=True)
Linear(in_features=1, out_features=5, bias=True)
5 -> 1
epoch : 1/2, loss = 1169026.916176
epoch : 2/2, loss = 1169000.477941


In [314]:
data.dtypes

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

In [315]:
new_train.dtypes

Store                               int64
DayOfWeek                           int64
Sales                               int64
Open                                int64
Promo                               int64
StateHoliday                        int64
SchoolHoliday                       int64
StoreType                           int64
Assortment                          int64
CompetitionDistance               float64
Promo2                              int64
Year                                int64
Month                               int64
Day                                 int64
CompetitionDays                   float64
PromoOpenDays                     float64
PromoInterval_0                     uint8
PromoInterval_Feb,May,Aug,Nov       uint8
PromoInterval_Jan,Apr,Jul,Oct       uint8
PromoInterval_Mar,Jun,Sept,Dec      uint8
dtype: object

In [316]:
new_test.dtypes

Id                                         int64
Store                                      int64
DayOfWeek                                  int64
Date                              datetime64[ns]
Open                                       int64
Promo                                      int64
StateHoliday                              object
SchoolHoliday                              int64
StoreType                                  int64
Assortment                                 int64
CompetitionDistance                      float64
CompetitionOpenSinceMonth                float64
CompetitionOpenSinceYear                 float64
Promo2                                     int64
Promo2SinceWeek                          float64
Promo2SinceYear                          float64
Year                                       int64
Month                                      int64
Day                                        int64
CompetitionDays                          float64
PromoOpenDays       

In [317]:
for test in new_test:
    exist = False
    for train in new_train:
        if train == test:
            exist = True
    
    if not exist:
        new_test = new_test.drop([test], axis=1)

print(len(new_test.dtypes))
print(len(new_train.dtypes)-1) # beacause Sales is y data

19
19


In [318]:
cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_test.replace(cleanup3, inplace=True)
new_test['StateHoliday'] = new_test['StateHoliday'].astype(int)

In [319]:
new_test.dtypes

Store                               int64
DayOfWeek                           int64
Open                                int64
Promo                               int64
StateHoliday                        int64
SchoolHoliday                       int64
StoreType                           int64
Assortment                          int64
CompetitionDistance               float64
Promo2                              int64
Year                                int64
Month                               int64
Day                                 int64
CompetitionDays                   float64
PromoOpenDays                     float64
PromoInterval_0                     uint8
PromoInterval_Feb,May,Aug,Nov       uint8
PromoInterval_Jan,Apr,Jul,Oct       uint8
PromoInterval_Mar,Jun,Sept,Dec      uint8
dtype: object

In [320]:
new_train.dtypes

Store                               int64
DayOfWeek                           int64
Sales                               int64
Open                                int64
Promo                               int64
StateHoliday                        int64
SchoolHoliday                       int64
StoreType                           int64
Assortment                          int64
CompetitionDistance               float64
Promo2                              int64
Year                                int64
Month                               int64
Day                                 int64
CompetitionDays                   float64
PromoOpenDays                     float64
PromoInterval_0                     uint8
PromoInterval_Feb,May,Aug,Nov       uint8
PromoInterval_Jan,Apr,Jul,Oct       uint8
PromoInterval_Mar,Jun,Sept,Dec      uint8
dtype: object

In [321]:
test = new_test.to_numpy()
y = feedforward(models, test)
arr = y.data.cpu().numpy()
prediction_data = pd.DataFrame(arr)
prediction_data

0
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
...    ...
35099  0.0
35100  0.0
35101  0.0
35102  0.0
35103  0.0

[35104 rows x 1 columns]

In [322]:
output = pd.DataFrame({'Id': test_data.Id, 'Sales': prediction_data[0]})
output.fillna(0, inplace=True)
output.to_csv('submission.csv', index=False)
output


Id  Sales
0          1    0.0
1          2    0.0
2          3    0.0
3          4    0.0
4          5    0.0
...      ...    ...
41083  41084    0.0
41084  41085    0.0
41085  41086    0.0
41086  41087    0.0
41087  41088    0.0

[41088 rows x 2 columns]